# Układy liniowe
Kody pomocniczne:
processArray - zwraca jednowymiarową tablicę z pliku parametry:
* arrname = nazwa zmiennej (wyjście jest kodem który trzeba wkleić żeby coś narysować)
* filename = nazwa pliku do wczytania
* start, end numer pierwszego i ostatniego wiersza do wczytania
* col = numer kolumny do wczytania
* dbg = flaga do debugowania

makePlotSim - rysuje wykres amplitudowy/częstotliwościowy :
* freq - tablica częstotliwości
* gain - tablica amplitudy
* phase - tablica fazy
* freqHz - częstotliwość graniczna (rysuje pinową linię)


makePlotSim - rysuje wykres amplitdowy/częstotliwości symulacje + dane

makeOscPlot - rysuje obraze z oscyloskopu
* t1, t2 - czasy z kanałów
* ch1, ch2 - sygnały z kanałów
* offset  - przesunięcie czasu (wykresy ładniej wyglądają jak się zaczynają od zera, nieprawdaż :)

## kody pomocnicze

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import matplotlib.ticker as ticker

def makePhasePlotSim(freq, gain, phase, freqHz=0., ):
    G = gain.copy()
    F = freq.copy()
    P = phase.copy()
    fig, axes = plt.subplots(2,1)
    fig.set_size_inches(8,8)
    axes[0].plot(F,G,"#ff0000")
    axes[1].plot(F,P,"#00ff00")
    axes[0].set_xscale('log')
    axes[1].set_xscale('log')
    axes[0].set_xlabel("freq (Hz)")
    axes[1].set_xlabel("freq (Hz)")
    axes[0].set_ylabel("Amp (dB)")
    axes[1].set_ylabel("phase (deg)")
    if freqHz!=0:
        axes[0].axvline(freqHz,color="b",linestyle="dotted")
        axes[1].axvline(freqHz,color="b",linestyle="dotted")
    axes[0].grid(True)
    axes[1].grid(True)

    plt.tight_layout()
    plt.show()
    
def makePhasePlotSim2(freq1, gain1, phase1,freq2, gain2, phase2):
  #  freq = processFile(filename, start_line,end_line, col=0)
  #  gain = processFile(filename, start_line,end_line, col=1)
  #  phase = processFile(filename, start_line,end_line, col=3,dbg=False)
    G1 = gain1.copy()
    G2 = gain2.copy()
    P1 = phase1.copy()
    P2 = phase2.copy()
    F1 = freq1.copy()
    F2 = freq2.copy()
    fig, axes = plt.subplots(2,1)
    fig.set_size_inches(8,8)
    axes[0].plot(F1,G1,"#ff0000",label="sim")
    axes[1].plot(F1,P1,"#ff0000",label="sim")
    axes[0].plot(F2,G2,"#00ff00",marker="o",label="data")
    axes[1].plot(F2,P2,"#00ff00",marker="o",label="data")
    axes[0].set_xscale('log')
    axes[1].set_xscale('log')
    axes[0].set_xlabel("freq (Hz)")
    axes[1].set_xlabel("freq (Hz)")
    axes[0].set_ylabel("Amp (dB)")
    axes[1].set_ylabel("phase (deg)")
    axes[0].grid(True)
    axes[1].grid(True)
    axes[1].legend(loc="lower right")
    axes[0].legend(loc="lower left")
    axes[1].grid(True)
    plt.tight_layout()
    plt.show()    
    

def makeOscPlot(t1, ch1, t2, ch2, offset = 0):
    fig, axes = plt.subplots(1,1)
    t1_copy = t1.copy() - offset
    t2_copy = t2.copy()-offset
    axes.plot(t1_copy,ch1,"#ff0000",label="ch1 (source)")
    axes.plot(t2_copy,ch2,"#0000ff",label="ch2 (response)")
    axes.set_xlabel("t (us)")
    axes.set_ylabel("U (V)")
    axes.legend(loc="lower right")
    plt.tight_layout()
    plt.show()

class osci_data:
    def __init__(self): 
        self.t1= None # czas 1 
        self.t2 = None
        self.ch1= None # kanal 1
        self.ch2 = None
    
class filter_dataset:
    def __init__(self):
        self.freq_data=None
        self.freq_sim=None
        self.phase_data=None
        self.phase_sim = None
        self.gain_data = None
        self.gain_sim = None
        self.freq_01_sim = osci_data()
        self.freq_10_sim = osci_data()
        self.freq_01_data = osci_data()
        self.freq_10_data = osci_data()

## Filtr dolnoprzepustowy:

Dane:

In [ ]:
# tutaj tworzymy wypełnione tablice



In [ ]:
Filter =  filter_dataset()

# tutaj sobie tworzymy wypełnione tablice 

# freq - czestotliwosci
# data - dane
# sim - symulacja
# gain - amplitudoa
# phase - faza
#freq_01 dane dla f = 0.1 fg
#freq_10 dane dla f = 10 fg

Filter.freq_sim = freq_sim
Filter.gain_sim = gain_sim
Filter.phase_sim = phase_sim

Filter.gain_data = gain_data
Filter.freq_data = freq_data
Filter.phase_data = phase_data

Filter.freq_01_sim.t1 = t1_01_sim 
Filter.freq_01_sim.t2 = t2_01_sim
Filter.freq_01_sim.ch1 = ch1_01_sim
Filter.freq_01_sim.ch2 = ch2_01_sim
Filter.freq_01_data.t1 = t1_01_data
Filter.freq_01_data.t2 = t2_01_data
Filter.freq_01_data.ch1 = ch1_01_data 
Filter.freq_01_data.ch2 = ch2_01_data

Filter.freq_10_sim.t1 = t1_10_sim
Filter.freq_10_sim.t2 = t2_10_sim
Filter.freq_10_sim.ch1 = ch1_10_sim
Filter.freq_10_sim.ch2 = ch2_10_sim
Filter.freq_10_data.t1 = t1_10_data
Filter.freq_10_data.t2 = t2_10_data
Filter.freq_10_data.ch1 = ch1_10_data 
Filter.freq_10_data.ch2 = ch2_10_data

### Symulacja
Wykres charektrystyki amplitudowej i fazowej.

In [ ]:
makePhasePlotSim(Filter.freq_sim, Filter.gain_sim, Filter.phase_sim)


Odpowiedź układu na sygnał prostokątny o częstotliwośi 0.1 fg

In [ ]:
makeOscPlot(Filter.freq_01_sim.t1,Filter.freq_01_sim.ch1,Filter.freq_01_sim.t2,Filter.freq_01_sim.ch2 ,offset=740000)

Odpowiedź układu na sygnał prostokątny o częstotliwośi 10 fg

In [ ]:
makeOscPlot(Filter.freq_10_sim.t1,Filter.freq_10_sim.ch1,Filter.freq_10_sim.t2,Filter.freq_10_sim.ch2 )

### Pomiar
Wykres charakterystyki fazowej i amplitudowej.

In [ ]:
makePhasePlotSim(Filter.freq_data, Filter.gain_data, Filter.phase_data)

Odpowiedź układu na sygnał prostokątny o częstotliwośi 0.1 fg

In [ ]:
makeOscPlot(Filter.freq_01_data.t1,Filter.freq_01_data.ch1,Filter.freq_01_data.t2,Filter.freq_01_data.ch2 )

Odpowiedź układu na sygnał prostokątny o częstotliwośi 10 fg

In [ ]:
makeOscPlot(Filter.freq_10_data.t1,Filter.freq_10_data.ch1,Filter.freq_10_data.t2,Filter.freq_10_data.ch2 )

### <font color='red'>Czas narastania:<font>
Dla częstotliwości dziesięciokrotnie mniejszej od częstotliwości granicznej policzyć czas narastania:

In [ ]:
t01 = 1
t09 = 0
t_narastania = t09-t01
print("czas narastania (symulacja): {0} {1}".format(t_narastania,"nazwa jednostki"))
t01 = 1
t09 = 0
t_narastania = t09-t01
print("czas narastania (dane): {0} {1}".format(t_narastania,"nazwa jednostki"))

### Porównania:
Porównanie symulacji z pomiarem charakterysytka fazowa i amplitudowa.

In [ ]:
makePhasePlotSim2(Filter.freq_sim, Filter.gain_sim, Filter.phase_sim,Filter.freq_data, Filter.gain_data, Filter.phase_data)

## Filtr górnoprzepustowy:

Dane:

In [ ]:
# tutaj tworzymy wypełnione tablice np.


In [ ]:
Filter =  filter_dataset()

# freq - czestotliwosci
# data - dane
# sim - symulacja
# gain - amplitudoa
# phase - faza
#freq_01 dane dla f = 0.1 fg
#freq_10 dane dla f = 10 fg

Filter.freq_sim = freq_sim
Filter.gain_sim = gain_sim
Filter.phase_sim = phase_sim

Filter.gain_data = gain_data
Filter.freq_data = freq_data
Filter.phase_data = phase_data

Filter.freq_01_sim.t1 = t1_01_sim 
Filter.freq_01_sim.t2 = t2_01_sim
Filter.freq_01_sim.ch1 = ch1_01_sim
Filter.freq_01_sim.ch2 = ch2_01_sim
Filter.freq_01_data.t1 = t1_01_data
Filter.freq_01_data.t2 = t2_01_data
Filter.freq_01_data.ch1 = ch1_01_data 
Filter.freq_01_data.ch2 = ch2_01_data

Filter.freq_10_sim.t1 = t1_10_sim
Filter.freq_10_sim.t2 = t2_10_sim
Filter.freq_10_sim.ch1 = ch1_10_sim
Filter.freq_10_sim.ch2 = ch2_10_sim
Filter.freq_10_data.t1 = t1_10_data
Filter.freq_10_data.t2 = t2_10_data
Filter.freq_10_data.ch1 = ch1_10_data 
Filter.freq_10_data.ch2 = ch2_10_data

### Symulacja
Wykres charektrystyki amplitudowej i fazowej:

In [ ]:
makePhasePlotSim(Filter.freq_sim, Filter.gain_sim, Filter.phase_sim)


Odpowiedź układu na sygnał prostokątny o częstotliwośi 0.1 fg

In [ ]:
makeOscPlot(Filter.freq_01_sim.t1,Filter.freq_01_sim.ch1,Filter.freq_01_sim.t2,Filter.freq_01_sim.ch2 )

Odpowiedź układu na sygnał prostokątny o częstotliwośi 10 fg

In [ ]:
makeOscPlot(Filter.freq_10_sim.t1,Filter.freq_10_sim.ch1,Filter.freq_10_sim.t2,Filter.freq_10_sim.ch2 )

### Pomiar
Wykres charakterystyki fazowej i amplitudowej, wraz z zaznaczoną częstotliwością graniczną.

In [ ]:
makePhasePlotSim(Filter.freq_data, Filter.gain_data, Filter.phase_data)

Odpowiedź układu na sygnał prostokątny o częstotliwośi 0.1 fg

In [ ]:
makeOscPlot(Filter.freq_01_data.t1,Filter.freq_01_data.ch1,Filter.freq_01_data.t2,Filter.freq_01_data.ch2 )

Odpowiedź układu na sygnał prostokątny o częstotliwośi 10 fg

In [ ]:
makeOscPlot(Filter.freq_10_data.t1,Filter.freq_10_data.ch1,Filter.freq_10_data.t2,Filter.freq_10_data.ch2 )

### <font color='red'>pomiary zwisu:</font>
Dokonać oszacowania zwisu dla częstotliwości 10 większej od fg.

In [ ]:
U1 = 1
U2 = 1
z = (U1-U2)/U1
print("zwis (symulacja): {0} %".format(z*100.0))
U1 = 1
U2 = 1
z = (U1-U2)/U1
print("zwis (dane): {0} %".format(z*100.0))

### Porównania:
Porównanie symulacji z pomiarem charakterysytka fazowa i amplitudowa:

In [ ]:
makePhasePlotSim2(Filter.freq_sim, Filter.gain_sim, Filter.phase_sim,Filter.freq_data, Filter.gain_data, Filter.phase_data)

## Nieskompensowany dzielnik napięcia:

Dane:

In [ ]:
Filter =  filter_dataset()


# freq - czestotliwosci
# data - dane
# sim - symulacja
# gain - amplitudoa
# phase - faza
#freq_01 dane dla f = 0.1 fg
#freq_10 dane dla f = 10 fg

Filter.freq_sim = freq_sim
Filter.gain_sim = gain_sim
Filter.phase_sim = phase_sim

Filter.gain_data = gain_data
Filter.freq_data = freq_data
Filter.phase_data = phase_data

Filter.freq_01_sim.t1 = t1_01_sim 
Filter.freq_01_sim.t2 = t2_01_sim
Filter.freq_01_sim.ch1 = ch1_01_sim
Filter.freq_01_sim.ch2 = ch2_01_sim
Filter.freq_01_data.t1 = t1_01_data
Filter.freq_01_data.t2 = t2_01_data
Filter.freq_01_data.ch1 = ch1_01_data 
Filter.freq_01_data.ch2 = ch2_01_data

Filter.freq_10_sim.t1 = t1_10_sim
Filter.freq_10_sim.t2 = t2_10_sim
Filter.freq_10_sim.ch1 = ch1_10_sim
Filter.freq_10_sim.ch2 = ch2_10_sim
Filter.freq_10_data.t1 = t1_10_data
Filter.freq_10_data.t2 = t2_10_data
Filter.freq_10_data.ch1 = ch1_10_data 
Filter.freq_10_data.ch2 = ch2_10_data

### Symulacja
Wykres charektrystyki amplitudowej i fazowej:

In [ ]:
makePhasePlotSim(Filter.freq_sim, Filter.gain_sim, Filter.phase_sim)


Odpowiedź układu na sygnał prostokątny o częstotliwośi 0.1 fg

In [ ]:
makeOscPlot(Filter.freq_01_sim.t1,Filter.freq_01_sim.ch1,Filter.freq_01_sim.t2,Filter.freq_01_sim.ch2 )

Odpowiedź układu na sygnał prostokątny o częstotliwośi 10 fg

In [ ]:
makeOscPlot(Filter.freq_10_sim.t1,Filter.freq_10_sim.ch1,Filter.freq_10_sim.t2,Filter.freq_10_sim.ch2 )

### Pomiar
Wykres charakterystyki fazowej i amplitudowej

In [ ]:
makePhasePlotSim(Filter.freq_data, Filter.gain_data, Filter.phase_data)

Odpowiedź układu na sygnał prostokątny o częstotliwośi 0.1 fg

In [ ]:
makeOscPlot(Filter.freq_01_data.t1,Filter.freq_01_data.ch1,Filter.freq_01_data.t2,Filter.freq_01_data.ch2 )

Odpowiedź układu na sygnał prostokątny o częstotliwośi 10 fg

In [ ]:
makeOscPlot(Filter.freq_10_data.t1,Filter.freq_10_data.ch1,Filter.freq_10_data.t2,Filter.freq_10_data.ch2 )

### Porównania:
Porównanie symulacji z pomiarem charakterysytka fazowa i amplitudowa:

In [ ]:
makePhasePlotSim2(Filter.freq_sim, Filter.gain_sim, Filter.phase_sim,Filter.freq_data, Filter.gain_data, Filter.phase_data)

## Skompensowany dzielnik napięcia:

Dane:

In [ ]:
Filter =  filter_dataset()

# freq - czestotliwosci
# data - dane
# sim - symulacja
# gain - amplitudoa
# phase - faza
#freq_01 dane dla f = 0.1 fg
#freq_10 dane dla f = 10 fg

Filter.freq_sim = freq_sim
Filter.gain_sim = gain_sim
Filter.phase_sim = phase_sim

Filter.gain_data = gain_data
Filter.freq_data = freq_data
Filter.phase_data = phase_data

Filter.freq_01_sim.t1 = t1_01_sim 
Filter.freq_01_sim.t2 = t2_01_sim
Filter.freq_01_sim.ch1 = ch1_01_sim
Filter.freq_01_sim.ch2 = ch2_01_sim
Filter.freq_01_data.t1 = t1_01_data
Filter.freq_01_data.t2 = t2_01_data
Filter.freq_01_data.ch1 = ch1_01_data 
Filter.freq_01_data.ch2 = ch2_01_data

Filter.freq_10_sim.t1 = t1_10_sim
Filter.freq_10_sim.t2 = t2_10_sim
Filter.freq_10_sim.ch1 = ch1_10_sim
Filter.freq_10_sim.ch2 = ch2_10_sim
Filter.freq_10_data.t1 = t1_10_data
Filter.freq_10_data.t2 = t2_10_data
Filter.freq_10_data.ch1 = ch1_10_data 
Filter.freq_10_data.ch2 = ch2_10_data

### Symulacja
Wykres charektrystyki amplitudowej i fazowej:

In [ ]:
makePhasePlotSim(Filter.freq_sim, Filter.gain_sim, Filter.phase_sim)


Odpowiedź układu na sygnał prostokątny o częstotliwośi 0.1 fg

In [ ]:
makeOscPlot(Filter.freq_01_sim.t1,Filter.freq_01_sim.ch1,Filter.freq_01_sim.t2,Filter.freq_01_sim.ch2 )

Odpowiedź układu na sygnał prostokątny o częstotliwośi 10 fg

In [ ]:
makeOscPlot(Filter.freq_10_sim.t1,Filter.freq_10_sim.ch1,Filter.freq_10_sim.t2,Filter.freq_10_sim.ch2 )

### Pomiar
Wykres charakterystyki fazowej i amplitudowej

In [ ]:
makePhasePlotSim(Filter.freq_data, Filter.gain_data, Filter.phase_data)

Odpowiedź układu na sygnał prostokątny o częstotliwośi 0.1 fg

In [ ]:
makeOscPlot(Filter.freq_01_data.t1,Filter.freq_01_data.ch1,Filter.freq_01_data.t2,Filter.freq_01_data.ch2 )

Odpowiedź układu na sygnał prostokątny o częstotliwośi 10 fg

In [ ]:
makeOscPlot(Filter.freq_10_data.t1,Filter.freq_10_data.ch1,Filter.freq_10_data.t2,Filter.freq_10_data.ch2 )

### Porównania:
Porównanie symulacji z pomiarem charakterysytka fazowa i amplitudowa:

In [ ]:
makePhasePlotSim2(Filter.freq_sim, Filter.gain_sim, Filter.phase_sim,Filter.freq_data, Filter.gain_data, Filter.phase_data)

## Warunek na kompensacje dzielnika napięcia
(wyprowadzenie i pojemność kondensatora C2)

### jak używać latex'a w Python:
1. przełączyć komórkę w tryb "markdown" (tj. tekstowy)
2. wpisać kod latex'a między dolarami przykład:
$ E = mc^{2}$
Typy:
1. Znacznik dzielenia frac np. x przez y to \frac{x}{y}, indeks góry ^ np. e^{x} = $ e^{x} $indeks dolny _ np u_{w} = $ u_{w} $
2. Jest taka strona detexify (tam się rysuje znaczek i ona wypluwa kod latexowy)

$\psi$

Częstotliwości graniczne - rachunek niepewności + wyniki (dla filtrów + nieskomensowanego dzielnika napięcia):

Częstotliwości graniczne - porównanie wyników z symulacjami: